In [282]:
#import libraries used into
from datetime import datetime, timedelta
import pandas as pd, numpy as np, os

In [283]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
print("fileam: ", fileam)
if datetime.today().strftime('%A') == "Monday":
    fileyest = "OT " + (datetime.today() - timedelta(days = 3)).strftime('%d.%m.%Y' + " 5 P.M..XLSX")
    print("fileyest: ", fileyest)
else:
    fileyest = "OT " + (datetime.today() - timedelta(days = 1)).strftime('%d.%m.%Y' + " 5 P.M..XLSX")
    print("fileyest: ", fileyest)
filepm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 5 P.M..XLSX"
print("filepm: ", filepm)
file3pm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 3 P.M..XLSX"
print("file3pm: ", file3pm)
main_path = "C:\\Users\\mruiz\\Documents\\DB OT\\" 
current_path = "C:\\Users\\mruiz\\Documents\\DB OT\\CurrentDocuments"
current_month = datetime.now().strftime('%B')
current_path_ = os.listdir(current_path)
main_path_ = os.listdir(main_path)
data_yesterday_excel = {}
data_am_excel = {}
data_3pm_excel = {}
data_pm_excel = {}
repro_nuevas = 1

fileam:  OT 10.11.2022.XLSX
fileyest:  OT 09.11.2022 5 P.M..XLSX
filepm:  OT 10.11.2022 5 P.M..XLSX
file3pm:  OT 10.11.2022 3 P.M..XLSX


PRUEBAS

Fin de las pruebas

In [284]:
def get_data(entry_data):
    file = main_path + entry_data
    filexcel = pd.read_excel(file,sheet_name="Sheet1")
    print("Los datos del archivo ", entry_data, " fueron procesados")
    return filexcel

In [285]:
while True:
    try:
        data_yesterday_excel = get_data(fileyest)
        data_am_excel = get_data(fileam)
        break
    except FileNotFoundError:
        print("Hay al menos un archivo que no fue leído: ")
        print("Archivo Ayer : ", type(data_yesterday_excel))
        print("Archivo hoy por la mañana: ", type(data_am_excel))
        break

Los datos del archivo  OT 09.11.2022 5 P.M..XLSX  fueron procesados
Los datos del archivo  OT 10.11.2022.XLSX  fueron procesados


In [286]:
if type(data_am_excel) == type(data_yesterday_excel):
    programa = data_am_excel
    print("Los datos de la mañana fueron seleccionados")
else :
    print("Error")

Los datos de la mañana fueron seleccionados


In [287]:
data2 =current_path + "\\Master_OT.xlsx"
nombres = pd.read_excel(data2, sheet_name="Names", header=0)

In [288]:
data3 = current_path + "\\Politicas.XLSX"
politicas = pd.read_excel(data3, sheet_name="POLITICAS CONTRATO", header=0)

In [289]:
data4 = current_path + "\\Materiales_Locales.xlsx"
mlocal = pd.read_excel(data4, sheet_name="ENTREGADO", header=0)

In [290]:
data5 = current_path + "\\Materiales_Foraneos.xlsx"
mforaneo = pd.read_excel(data5, sheet_name="MATRIZ FINAL", header=1)

In [291]:
data6 = current_path + "\\Notificadas_Mensual.xlsx"
reportes = pd.read_excel(data6, sheet_name="CONTROL_REMASTERED", header=0)

In [292]:
data7 = current_path + "\\Ubicaciones_Equipos.xlsx"
ubicaciones = pd.read_excel(data7, sheet_name="Sheet1", header=0)

In [293]:
data8 = current_path + "\\Reprogramaciones.xlsx"
reprogramaciones_ = pd.read_excel(data8, sheet_name="DB", header=0)

C:\Users\mruiz\AppData\Local\Temp\ipykernel_17432\1808906021.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  reprogramaciones_ = pd.read_excel(data8, sheet_name="DB", header=0)


In [294]:
reprogramaciones = data_yesterday_excel.copy()

In [295]:
reprogramaciones.rename(columns={'Inicio programado':'Fecha_reprogramada'}, inplace=True)

In [296]:
programa['Material'] = np.where((programa['Clase de orden'] == 'ZMCP') &  (programa['Texto breve'].str.contains('Emer|Revi', case=False)), 'NO',
np.where(programa['Clase de orden'] == 'ZPM2','SI',
np.where(programa['Clase de orden'] == 'ZMCP','SI',
np.where(programa['Clase de orden'] == 'ZMCI', 'SI',
np.where((programa['Clase de orden'] == 'ZPM1') &  (programa['Texto breve'].str.contains('CAFA|CABAT|ACUM', case=False)), 'SI','NO')))))

In [297]:
#print('Las columnas de frame 1 son', programa.info())
#print('Las columnas de frame 2 son', nombres.columns)
#print('Las columnas de frame 3 son', politicas.columns)
#print('Las columnas de frame 4 son', mlocal.columns)
#print('Las columnas de frame 5 son', mforaneo.columns)
#print('Las columnas de reprogramaciones son ',reprogramaciones)
#print('Las columnas de frame 6 son', reportes.columns)
#print('Las columnas de frame 7 son', ubicaciones.columns)
#print('Las columnas de frame main son', result_df.columns)

In [298]:
reportes['Inicio programado'] = reportes['Inicio programado'].str.replace("-","/")
reportes['Inicio programado'] = pd.to_datetime(reportes['Inicio programado'], format = "%d/%m/%Y", exact=False)
print(reportes['Inicio programado'].dtype)

datetime64[ns]


In [299]:
result_df = pd.merge(left = programa, right = nombres[['TecID','Short name','Estatus','City']],left_on="Pto.tbjo.responsable",right_on="TecID",how="left")

In [300]:
result_df = pd.merge(result_df,politicas[['Equipo','gestora archivo anahi ','Frecuencia',
       'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
       'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc',
       'Tiempo valido de prueba', 'Tecnicos para servicio',
       'Requerimientos adicionales', 'Correo solicitud acceso',
       'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales',
       'OC', 'FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS']],on="Equipo",how="left")

In [301]:
result_df = pd.merge(result_df,mlocal[['Orden', 'Inicio programado2', 'Fecha de salida',
       'Comentario', 'Material Separado']], on=['Orden'],how= "left")

In [302]:
result_df = pd.merge(result_df,mforaneo[['Orden','UBICACIÓN', 'ESTATUS COMPRA', 'PROVEEDOR']],on='Orden',how='left')

In [303]:
result_df = pd.merge(left = result_df,right = reprogramaciones[['Orden','Fecha_reprogramada']],on='Orden',how='left')

In [304]:
result_df = pd.merge(result_df,reportes,on=['Inicio programado','Orden','Pto.tbjo.responsable','Cliente','Denominación de objeto técnico',
'Número de serie','Equipo','Clase de orden','OC'],how="outer")

C:\Users\mruiz\AppData\Local\Temp\ipykernel_17432\4216254623.py:1: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  result_df = pd.merge(result_df,reportes,on=['Inicio programado','Orden','Pto.tbjo.responsable','Cliente','Denominación de objeto técnico',


In [305]:
result_df = pd.merge(left=result_df,right=ubicaciones[['Equipo','Dirección']],how='left',on='Equipo')

In [306]:
print(result_df.loc[1])

Inicio programado                                                  2023-12-12 00:00:00
Orden                                                                        4093006.0
Plan mant.preventivo                                                             626.0
Pto.tbjo.responsable                                                          TECSER17
Status del sistema                                                             ABIERTA
                                                           ...                        
Gestora.1                                                                          NaN
preventivos                                                                        NaN
correctivos                                                                        NaN
comentarios internos adiccionales                                                  NaN
Dirección                            Av. Miguel Alemán 8789, Miguel Alemán, 66447 S...
Name: 1, Length: 102, dtype: object


In [307]:
result_df.assign(Pendiente_Recoger=0)

,Inicio programado,Orden,Plan mant.preventivo,Pto.tbjo.responsable,Status del sistema,Texto breve,Cliente,Denominación de objeto técnico,Número de serie,Equipo,...,SOLO AMARILLO,REPORTE INCOMPLETO,LEVANTAMIENTO,CONTRA RECIBO,Gestora.1,preventivos,correctivos,comentarios internos adiccionales,Dirección,Pendiente_Recoger
0,2024-07-26,4096274.0,8396.0,TECSER24,BLOQUEADA,SEGUIDORES DE LA CRUZ 10KW,303371.0,SEGUIDORES DE LA CRUZ 10KW,NaN,12923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Brecha de Las Anacuas, El Oasis, 88796 Reynosa...",0
1,2023-12-12,4093006.0,626.0,TECSER17,ABIERTA,HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA,300105.0,HD MIGUEL ALEMAN 8789 PTA 350KW,43203341,10373.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Av. Miguel Alemán 8789, Miguel Alemán, 66447 S...",0
2,2023-10-12,4093001.0,439.0,TECSER6,ABIERTA,HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL,300105.0,HD PIEDRAS NEGRAS 8638 PTA 350KW,2012658048,10270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Carretera 57 y Eulalio Gutiérrez, 26070 Piedra...",0
3,2023-09-13,4093222.0,1522.0,TECSER2,ABIERTA,HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR,300105.0,HD INTERLOMAS 8762 PTA 250KW,14808,10935.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Blvd. Magnocentro 35-Local SA - 4, Parques de ...",0
4,2023-09-11,4093049.0,653.0,TECSER6,ABIERTA,HD SAN PEDRO 8652 PTA 350KW ACUMULADOR,300105.0,HD SAN PEDRO 8652 PTA 350KW,UI4182W,10388.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Av. Eugenio Garza Lagüera 101, Valle Oriente, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45447,2022-10-28,4089302,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL CAFAM,FGWPEPPA,10082,...,NaN,NaN,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",0
45448,2022-10-28,4094159,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL 30 KW PREVENT MTY,91115251,10504,...,NaN,NaN,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",0
45449,2022-10-28,4089377,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL CAFAM,91115251,10504,...,NaN,NaN,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",0
45450,2022-10-27,4100651,NaN,TECSER16,NaN,NaN,NaN,PRIMER VISITA FRANCISCO VELAZCO REVISIO,NaN,NaN,...,NaN,NaN,NaN,NaN,Karla,NaN,NaN,NaN,NaN,0


In [308]:
result_df['Pendiente_Recoger'] = np.where(result_df['Inicio programado2'].isnull(),result_df['ESTATUS COMPRA'],result_df['Inicio programado2'])

In [309]:
result_df.assign(reprogramacion=0)

,Inicio programado,Orden,Plan mant.preventivo,Pto.tbjo.responsable,Status del sistema,Texto breve,Cliente,Denominación de objeto técnico,Número de serie,Equipo,...,REPORTE INCOMPLETO,LEVANTAMIENTO,CONTRA RECIBO,Gestora.1,preventivos,correctivos,comentarios internos adiccionales,Dirección,Pendiente_Recoger,reprogramacion
0,2024-07-26,4096274.0,8396.0,TECSER24,BLOQUEADA,SEGUIDORES DE LA CRUZ 10KW,303371.0,SEGUIDORES DE LA CRUZ 10KW,NaN,12923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Brecha de Las Anacuas, El Oasis, 88796 Reynosa...",NaN,0
1,2023-12-12,4093006.0,626.0,TECSER17,ABIERTA,HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA,300105.0,HD MIGUEL ALEMAN 8789 PTA 350KW,43203341,10373.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Av. Miguel Alemán 8789, Miguel Alemán, 66447 S...",NaN,0
2,2023-10-12,4093001.0,439.0,TECSER6,ABIERTA,HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL,300105.0,HD PIEDRAS NEGRAS 8638 PTA 350KW,2012658048,10270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Carretera 57 y Eulalio Gutiérrez, 26070 Piedra...",NaN,0
3,2023-09-13,4093222.0,1522.0,TECSER2,ABIERTA,HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR,300105.0,HD INTERLOMAS 8762 PTA 250KW,14808,10935.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Blvd. Magnocentro 35-Local SA - 4, Parques de ...",NaN,0
4,2023-09-11,4093049.0,653.0,TECSER6,ABIERTA,HD SAN PEDRO 8652 PTA 350KW ACUMULADOR,300105.0,HD SAN PEDRO 8652 PTA 350KW,UI4182W,10388.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Av. Eugenio Garza Lagüera 101, Valle Oriente, ...",NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45447,2022-10-28,4089302,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL CAFAM,FGWPEPPA,10082,...,NaN,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",NaN,0
45448,2022-10-28,4094159,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL 30 KW PREVENT MTY,91115251,10504,...,NaN,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",NaN,0
45449,2022-10-28,4089377,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL CAFAM,91115251,10504,...,NaN,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",NaN,0
45450,2022-10-27,4100651,NaN,TECSER16,NaN,NaN,NaN,PRIMER VISITA FRANCISCO VELAZCO REVISIO,NaN,NaN,...,NaN,NaN,NaN,Karla,NaN,NaN,NaN,NaN,NaN,0


In [310]:
reprogramaciones = result_df.copy()

In [311]:
result_df['reprogramacion'] = np.where(result_df['Inicio programado']==reprogramaciones['Fecha_reprogramada'], "Sin_Reprogramar","Reprogramada")
result_df['reprogramacion'] = np.where(result_df['Fecha_reprogramada'].isnull(),"O.T. Agregada",result_df['reprogramacion'])

In [312]:
print(result_df['reprogramacion'].unique())

['Sin_Reprogramar' 'O.T. Agregada' 'Reprogramada']


In [313]:
result_df.assign(Politica=0)

,Inicio programado,Orden,Plan mant.preventivo,Pto.tbjo.responsable,Status del sistema,Texto breve,Cliente,Denominación de objeto técnico,Número de serie,Equipo,...,LEVANTAMIENTO,CONTRA RECIBO,Gestora.1,preventivos,correctivos,comentarios internos adiccionales,Dirección,Pendiente_Recoger,reprogramacion,Politica
0,2024-07-26,4096274.0,8396.0,TECSER24,BLOQUEADA,SEGUIDORES DE LA CRUZ 10KW,303371.0,SEGUIDORES DE LA CRUZ 10KW,NaN,12923.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"Brecha de Las Anacuas, El Oasis, 88796 Reynosa...",NaN,Sin_Reprogramar,0
1,2023-12-12,4093006.0,626.0,TECSER17,ABIERTA,HD MIGUEL ALEMAN 8789 PTA 350 KW ACUMULA,300105.0,HD MIGUEL ALEMAN 8789 PTA 350KW,43203341,10373.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"Av. Miguel Alemán 8789, Miguel Alemán, 66447 S...",NaN,Sin_Reprogramar,0
2,2023-10-12,4093001.0,439.0,TECSER6,ABIERTA,HD PIEDRAS NEGRAS 8638 PTA 350 KW ACUMUL,300105.0,HD PIEDRAS NEGRAS 8638 PTA 350KW,2012658048,10270.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"Carretera 57 y Eulalio Gutiérrez, 26070 Piedra...",NaN,Sin_Reprogramar,0
3,2023-09-13,4093222.0,1522.0,TECSER2,ABIERTA,HD INTERLOMAS 8762 PTA 250 KW ACUMULADOR,300105.0,HD INTERLOMAS 8762 PTA 250KW,14808,10935.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"Blvd. Magnocentro 35-Local SA - 4, Parques de ...",NaN,Sin_Reprogramar,0
4,2023-09-11,4093049.0,653.0,TECSER6,ABIERTA,HD SAN PEDRO 8652 PTA 350KW ACUMULADOR,300105.0,HD SAN PEDRO 8652 PTA 350KW,UI4182W,10388.0,...,NaN,NaN,NaN,NaN,NaN,NaN,"Av. Eugenio Garza Lagüera 101, Valle Oriente, ...",NaN,Sin_Reprogramar,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45447,2022-10-28,4089302,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL CAFAM,FGWPEPPA,10082,...,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",NaN,O.T. Agregada,0
45448,2022-10-28,4094159,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL 30 KW PREVENT MTY,91115251,10504,...,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",NaN,O.T. Agregada,0
45449,2022-10-28,4089377,NaN,TECSER19,NaN,NaN,300047,IPADE BUSSINES SCHOOL CAFAM,91115251,10504,...,NaN,NaN,Karla,NaN,NaN,NaN,"Gustavo Díaz Ordaz 100, Santa María, 64650 Mon...",NaN,O.T. Agregada,0
45450,2022-10-27,4100651,NaN,TECSER16,NaN,NaN,NaN,PRIMER VISITA FRANCISCO VELAZCO REVISIO,NaN,NaN,...,NaN,NaN,Karla,NaN,NaN,NaN,NaN,NaN,O.T. Agregada,0


In [314]:
result_df['Politica'] = np.where(result_df['Requiere Acceso'].isnull(),"NO","SI")

In [315]:
print(result_df.shape[0])
print(programa.shape[0])

45452
40740


In [326]:
reprogramaciones = result_df[result_df['reprogramacion'].isin(['Reprogramada'])]
if datetime.today().strftime('%Y-%m-%d') == reprogramaciones_['Dia_reprogramado'].loc[0]:
       repro_nuevas = 0
       print("Sin Reprogramaciones nuevas")
else:
       reprogramaciones = reprogramaciones.assign(Dia_reprogramado = datetime.today().strftime('%Y-%m-%d'))
       reprogramaciones = pd.concat([reprogramaciones,reprogramaciones_])
       print(reprogramaciones.shape[0])

182


In [318]:
reprogramaciones.columns

Index(['Inicio programado', 'Orden', 'Plan mant.preventivo',
       'Pto.tbjo.responsable', 'Status del sistema', 'Texto breve', 'Cliente',
       'Denominación de objeto técnico', 'Número de serie', 'Equipo',
       ...
       'CONTRA RECIBO ', 'Gestora.1', 'preventivos', 'correctivos',
       'comentarios internos adiccionales', 'Dirección', 'Pendiente_Recoger',
       'reprogramacion', 'Politica', 'Dia_reprogramado'],
      dtype='object', length=106)

In [319]:
listr = np.where(reportes.columns.str.contains('ESTATUS'), reportes.columns,"")
i = 0
eliminate = []
for element in listr:
    if element == "" : 
        eliminate.append(i)
    i = i+1
listr = np.delete(listr,eliminate)
estatus_actual = listr[-1]
listr = np.delete(listr,np.where(listr == estatus_actual))

In [320]:
listr_ = ['COMENTARIO ','Tipo Cliente','MES ','Mes2','Antiguedad','Gestora','Gpo Cliente','FORMATO ESPECIAL',
'SEMANA ANTERIORES ','FACTURA ','SOLO AMARILLO ','REPORTE INCOMPLETO ','LEVANTAMIENTO ','CONTRA RECIBO ',
'OC','Gestora','preventivos','correctivos','comentarios internos adiccionales','Gestora.1','TecID']
mylist = np.concatenate((listr,listr_),axis=None)
#reportes = reportes.drop(mylist,axis=1)

In [321]:
reprogramaciones = reprogramaciones[['Orden', 'Inicio programado','Plan mant.preventivo', 'Pto.tbjo.responsable',
       'Status del sistema', 'Texto breve', 'Cliente', 'Fecha inicio real',
       'Fecha_reprogramada','Dia_reprogramado']]

In [322]:
result_df = result_df.drop(mylist,axis=1)

In [323]:
if repro_nuevas == 0 :
    print("Se conservó el documento")
elif repro_nuevas != 0 : 
    os.remove(current_path + "\\" + "Reprogramaciones.xlsx")
    path = current_path + "\\" + "Reprogramaciones.xlsx"   
    reprogramaciones.to_excel(path, index=False,header=True)
    with pd.ExcelWriter(path) as writer:
        reprogramaciones.to_excel(writer,index=False,sheet_name="DB")
    os.startfile(path)

In [324]:
os.remove(current_path + "\\" + "Main.xlsx")
path = current_path + "\\" + "Main.xlsx"   
result_df.to_excel(path, index=False,header=True)
with pd.ExcelWriter(path) as writer:
    result_df.to_excel(writer,index=False,sheet_name="DB")
os.startfile(path)